In [1]:
#import libraries
try:
    # For Python 3.0 and later
    from urllib.request import urlopen
except ImportError:
    # Fall back to Python 2's urllib2
    from urllib2 import urlopen
import string
import json
import itertools 
from selenium import webdriver
from selenium.common.exceptions import NoSuchElementException
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup
import pandas as pd
import re

In [2]:
def check_exists_by_xpath(xpath):
    try:
        driver.find_element_by_xpath(xpath)
    except NoSuchElementException:
        return False
    return True

In [3]:
#starting a webdriver
driver = webdriver.Firefox()
timeout = 10
driver.get('https://racing.hkjc.com/racing/information/English/Horse/SelectHorse.aspx')

In [4]:
alphabets = string.ascii_uppercase
big_df_list = []

for i in alphabets: 
    #click on index
    driver.find_element_by_link_text(i).click()

    #find the size of the table and iterate through alphabet
    row_count = len(driver.find_elements_by_xpath("/html/body/div/p/table/tbody/tr[2]/td/table/tbody/tr"))
    column_count = len(driver.find_elements_by_xpath("/html/body/div/p/table/tbody/tr[2]/td/table/tbody/tr[1]/td"))

    for j in range(1,row_count+1):
        for k in range(1,column_count+1):
            # click on the letter             
            if(check_exists_by_xpath("/html/body/div/p/table/tbody/tr[2]/td/table/tbody/tr[%d]/td[%d]/table/tbody/tr/td[1]/li/a" %(j,k))):
                driver.find_element_by_xpath("/html/body/div/p/table/tbody/tr[2]/td/table/tbody/tr[%d]/td[%d]/table/tbody/tr/td[1]/li/a" %(j,k)).click() 
              
                # wait for the page to load
                try:
                    element = WebDriverWait(driver, 20).until(
                        EC.presence_of_element_located((By.XPATH, "/html/body/div[@class='commContent']/div[1]/table[@class='bigborder']"))
                    )
                except:
                    print('timed out')
                    
                # click on the "all seasons tab"
                if(check_exists_by_xpath("/html/body/div/div[1]/table[2]/tbody/tr/td[2]/table/tbody/tr/td[3]/img")):
                    driver.find_element_by_xpath("/html/body/div/div[1]/table[2]/tbody/tr/td[2]/table/tbody/tr/td[3]/img").click()
                
                # wait for the page to load
                try:
                    element = WebDriverWait(driver, 20).until(
                        EC.presence_of_element_located((By.XPATH, "/html/body/div[@class='commContent']/div[1]/table[@class='bigborder']"))
                    )
                except:
                    print('timed out')
                
                # grab source code
                df_list_all=pd.read_html(driver.page_source)
                df_performance_table=pd.read_html(driver.page_source, attrs={"class": "bigborder"})
                
                # get past performance table, set first row to header, drop video replay column, remove blank rows
                df = df_performance_table[0]
                df = df.rename(columns=df.iloc[0], inplace = False)
                df = df.drop(['VideoReplay'], axis=1, inplace=False)
                df = df.dropna(axis=0, how='any', thresh=None, subset=None, inplace=False)
                df = df[~df.RaceIndex.str.contains("Season")]
                df = df[~df.RaceIndex.str.contains("No Running Records")]
                df = df.drop([0], inplace = False)
                
                # create horse_id column
                s = df_list_all[1].iloc[0][0]
                horse_id= re.search('\(([^)]+)', s).group(1) 
                df.insert(loc=0, column='horse_id', value=horse_id)
                
                # append past performances to list of dataframes
                big_df_list.append(df)
                print("Collected data for: " + s)
                driver.back()                            
            else:
                break

    driver.back()

Collected data for: AETHERO (C336)
Collected data for: AMAZING STAR (B019)
Collected data for: ASSIMILATE (C160)
Collected data for: AMAZING ONE PLUS (D173)
Collected data for: AMAZING CHOCOLATE (C059)
Collected data for: ARTHUR'S KINGDOM (E194)
Collected data for: ALLIED AGILITY (C314)
Collected data for: ALL IN MIND (D200)
Collected data for: ALPHA HEDGE (B102)
Collected data for: AMAZING BEATS (C507)
Collected data for: ARMANDO (D351)
Collected data for: ABLE REIGN (E202)
Collected data for: APACHE PASS (D433)
Collected data for: ALL FOR SOUTH (D395)
Collected data for: AMARA WIN (D455)
Collected data for: ARROGANT (D025)
Collected data for: ALCARI (A354)
Collected data for: AMAZING NEWS (E123)
Collected data for: ABOVE (B345)
Collected data for: ANGEL OF MY EYES (D360)
Collected data for: AI ONE (E139)
Collected data for: AUTHENTIC CHAMP (C039)
Collected data for: AQUILA (C248)
Collected data for: AWE (D168)
Collected data for: ARES (D125)
Collected data for: AURORA PEGASUS (C351)


Collected data for: COURAGEOUS KNIGHT (E163)
Collected data for: CASA KINGDOM (E148)
Collected data for: CHIRON (E151)
Collected data for: CROWN AVENUE (V378)
Collected data for: CIGARY (D512)
Collected data for: CLUB LAD (D370)
Collected data for: COBY OPPA (C192)
Collected data for: COUNTRY BOY (D348)
Collected data for: CHAMPION PRIDE (B049)
Collected data for: CIGAR BUDDIES (D299)
Collected data for: COURAGEOUS DRAGON (C015)
Collected data for: CALIFORNIA LEGEND (B363)
Collected data for: COPARTNER ERA (C534)
Collected data for: CARRY THE DIAMOND (D122)
Collected data for: CHEERFUL STAR (B326)
Collected data for: CITY LEGEND (A086)
Collected data for: CIRCUIT NUMBER ONE (B373)
Collected data for: CRUCIATUS (C206)
Collected data for: CAN'T TOUCH THIS (D255)
Collected data for: CHUNGHWA JINGSHEN (D148)
Collected data for: CENTENARY IBIS (D163)
Collected data for: COOL ACE (D154)
Collected data for: CHARITYDREAM (B334)
Collected data for: CALIFORNIA SPANGLE (E314)
Collected data for: 

Collected data for: GORYTUS (E302)
Collected data for: GIFT OF LIFELINE (C172)
Collected data for: GOOD VIEW CLARICO (C454)
Collected data for: GUNNAR (C222)
Collected data for: GOD OF DRAGON (B108)
Collected data for: GOOD HEALTH (E159)
Collected data for: GREAT TREASURE (T380)
Collected data for: GLORIFIED FIRE (D418)
Collected data for: GOLFMAN STAR (D384)
Collected data for: GOOD BEAUTY (B384)
Collected data for: GREEN REIGN (D140)
Collected data for: GAMEPLAYER TIMES (D269)
Collected data for: GOOD LUCK CHARM (D402)
Collected data for: GLORIOUS LOVER (D191)
Collected data for: GOLDEN LINK (D222)
Collected data for: GALACTIC (D238)
Collected data for: GALLANT CROWN (D359)
Collected data for: GOODLUCK GOODLUCK (B188)
Collected data for: GOOD SHOT (C364)
Collected data for: GLENEALY GENERALS (B475)
Collected data for: GRATWICK (B110)
Collected data for: GALLANT CAVALIER (E238)
Collected data for: GALLANT EXPRESS (D442)
Collected data for: GEM OF SOUTH CHINA (E038)
Collected data for:

Collected data for: KING'S CAPITAL (E077)
Collected data for: KEEP YOU WARM (D272)
Collected data for: KING DRAGON (C437)
Collected data for: KABI SMART (E022)
Collected data for: KA YING SPIRIT (E217)
Collected data for: KILLER BEE (E297)
Collected data for: KING'S TROOPER (C130)
Collected data for: KOWLOON EAST STAR (E158)
Collected data for: KING'S HORN (D325)
Collected data for: KIRAM (T093)
Collected data for: KELMIMI WINS (C419)
Collected data for: KUNG FU TEA (C136)
Collected data for: KA FORTUNE (C421)
Collected data for: KUNGFUMASTER PANDA (C153)
Collected data for: KWAI CHUNG ELITE (C173)
Collected data for: KA YING BRILLIANCE (B115)
Collected data for: KEEP WINNING (B331)
Collected data for: LUCKY EXPRESS (D026)
Collected data for: LUCKY PATCH (D422)
Collected data for: LAKESHORE EAGLE (B391)
Collected data for: LUCKY MORE (D035)
Collected data for: LOBO'S LEGEND (C299)
Collected data for: LOCKHEED (B087)
Collected data for: LOVING A BOOM (C122)
Collected data for: LARSON (D

Collected data for: PEGASUS GLORY (D337)
Collected data for: PLEASANT ENDEAVOR (E114)
Collected data for: POISED TO STRIKE (D365)
Collected data for: PAKISTAN FRIEND (B370)
Collected data for: PRANCE DRAGON (B349)
Collected data for: PLAY WISE (A411)
Collected data for: PACKING AWARD (E264)
Collected data for: PANZERSCHRECK (E102)
Collected data for: PEACH BLOSSOMS (D202)
Collected data for: PEAK TO PEAK (D472)
Collected data for: PHENOMENAL COMBO (D493)
Collected data for: POLYMER GLORIOUS (E250)
Collected data for: POWER PROFIT (D315)
Collected data for: PRIDE OF DRAGON (E283)
Collected data for: PRIME STAR (E239)
Collected data for: PICHU (E011)
Collected data for: PACKING CHAMPION (D291)
Collected data for: PEARL CHAMPION (B201)
Collected data for: PARTY GENIUS (D217)
Collected data for: PAKISTAN ZINDABAD (D312)
Collected data for: PERFECT TO PLAY (B095)
Collected data for: PLIKCLONE (D003)
Collected data for: PRECISION DREAM (D410)
Collected data for: PRECIOUS SWEETIE (B254)
Colle

Collected data for: SEA MONARCH (D063)
Collected data for: SKY CHAMPION (E010)
Collected data for: SNOW AIRJET (D094)
Collected data for: SPARKLING STAR (D007)
Collected data for: SPARKY STAR (C416)
Collected data for: STAR OF WUYI (D503)
Collected data for: SHADOW BREAKER (C268)
Collected data for: SUPER HONG KONG (D241)
Collected data for: SMART HUNTER (D439)
Collected data for: SUNNY AND GOLD (D177)
Collected data for: SMART COUSIN (D249)
Collected data for: SPLIT OF A SECOND (B027)
Collected data for: SPONTANEOUS (D106)
Collected data for: STAY CHILL (D189)
Collected data for: STUNNING IMPACT (C366)
Collected data for: SUNDAY BETTING (E064)
Collected data for: SUPER BULLET (D001)
Collected data for: SHANGHAI GRACE (D190)
Collected data for: SMART WONGCHOY (D208)
Collected data for: SMILING FACE (D192)
Collected data for: SUPREME PLUS (C252)
Collected data for: SWOT TROOPERS WIND (D101)
Collected data for: SCENIC SWEETIE (D437)
Collected data for: SHANGHAI DRAGON (B114)
Collected da

Collected data for: WINNER METHOD (E178)
Collected data for: WINNING FEELING (C324)
Collected data for: WIND GLIDER (C433)
Collected data for: WISDOM PATCH (D289)
Collected data for: WOOD ON FIRE (C460)
Collected data for: WHISKEY NEAT (D150)
Collected data for: WAH MAY LUCK (E086)
Collected data for: WHO LOVES THE ACE (E267)
Collected data for: WINMORE STAR (E342)
Collected data for: WINNING DELAGO (E289)
Collected data for: WINNING GLORY (E333)
Collected data for: WIN TECH (E152)
Collected data for: WIN WIN STAR (E006)
Collected data for: WONDERFUL FANTASY (E326)
Collected data for: WINNING STEED (D155)
Collected data for: WON BALL (C504)
Collected data for: WINS ALL (C388)
Collected data for: WINNING VOLATILITY (D499)
Collected data for: WEALTHY EMPLOYEE (D252)
Collected data for: WAYFOONG STAR (D114)
Collected data for: WINNER'S HEART (D044)
Collected data for: WIN FOR YOUTH (D046)
Collected data for: WONDER EXPRESS (C224)
Collected data for: WINWIN THIRTYTHREE (C110)
Collected dat

In [5]:
big_df = pd.concat(big_df_list)

In [6]:
big_df.to_csv('past_performances.csv')